In [1]:
from data_dictionary import query_dataDict
from fileUtils import fileDownload
from sdmx import sdmx_struc

#### Read from data_dictionary all API sources (so far only two among all indicators populated)

In [2]:
# path to excel data dictionary in repo
data_dict_file = './data_dictionary/indicator_dictionary_TM_v1.xlsx'

In [3]:
# get indicators that are extracted by API (code and address in dataframe)
api_code_addr_df = query_dataDict.get_API_code_address(data_dict_file)
api_code_addr_df

,Code,Address
5,DM_BRTS,https://sdmx.data.unicef.org/ws/public/sdmxapi...
16,DM_POP_URBN,https://sdmx.data.unicef.org/ws/public/sdmxapi...
17,DM_FRATE_TOT,https://sdmx.data.unicef.org/ws/public/sdmxapi...
18,MNCH_ABR,https://sdmx.data.unicef.org/ws/public/sdmxapi...
19,NT_BW_LBW,https://sdmx.data.unicef.org/ws/public/sdmxapi...
20,CME_MRY0,https://sdmx.data.unicef.org/ws/public/sdmxapi...
21,CME_MRY0T4,https://sdmx.data.unicef.org/ws/public/sdmxapi...
22,CME_MRM0,https://sdmx.data.unicef.org/ws/public/sdmxapi...
23,MNCH_MMR,https://sdmx.data.unicef.org/ws/public/sdmxapi...
24,HVA_PMTCT_ARV_CVG,https://sdmx.data.unicef.org/ws/public/sdmxapi...


#### Get Unicef DataFlow TransMonEE country list

In [4]:
# UNICEF’s REST API data endpoint for TransMonEE Dataflow
url_endpoint = 'https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/ECARO,TRANSMONEE,1.0/'

In [5]:
# address for dataflow structure request
api_address = url_endpoint + 'all'
# parameters for dataflow structure request
api_params = {'format':'sdmx-json', 'detail':'structureOnly'}

# API request (dataflow structure)
d_flow_struc = fileDownload.api_request(api_address,api_params)

In [6]:
# all TransMonEE country codes are taken from its dataflow (requested by Eduard)
country_codes = sdmx_struc.get_all_country_codes(d_flow_struc.json())
# Join string of all countries for SDMX requests
country_call = '+'.join(country_codes.values())

#### Loop pandas dataframe `api_code_addr_df` and download raw data for all API sources therein

In [7]:
# raw data destination path
dest_path = './data_raw/'

In [8]:
# this is a loop along the `api_code_addr_df` pandas dataframe
for index, row in api_code_addr_df.iterrows():
    
    url_endpoint = row['Address']
    indicator_code = row['Code']
    
    print(f"Dealing with indicator: {indicator_code}")
    
    # first get dataflow number of dimensions
    api_address = url_endpoint + 'all'
    api_params = {'format':'sdmx-json', 'detail':'structureOnly'}
    d_flow_struc = fileDownload.api_request(api_address,api_params)
    n_dim = sdmx_struc.get_sdmx_dim(d_flow_struc.json())
    
    # prepare to dowload data (assumes all dataflows have the same country codes)
    api_address = url_endpoint + country_call + '.' + indicator_code + '.'*(n_dim-2)
    api_params = {'startPeriod':'1950', 'endPeriod':'2050', 'locale':'en'}
    api_headers = {'Accept':'application/vnd.sdmx.data+csv;version=1.0.0', 'Accept-Encoding':'gzip'}
    
    # now request indicator raw data
    indicator_raw = fileDownload.api_request(api_address,api_params,api_headers)
    
    # This flow could be improved if there are files already downloaded
    # In addition, maybe file names could include the year of execution
    
    # write raw data to destination file if requests satisfactory
    if indicator_raw.status_code == 200:
        dest_file = f"{dest_path}{indicator_code}.csv"
        with open(dest_file, 'wb') as f:
            f.write(indicator_raw.content)
        print(f"Indicator succesfully downloaded")

Dealing with indicator: DM_BRTS
Indicator succesfully downloaded
Dealing with indicator: DM_POP_URBN
Indicator succesfully downloaded
Dealing with indicator: DM_FRATE_TOT
Indicator succesfully downloaded
Dealing with indicator: MNCH_ABR
HTTP error occurred: 404 Client Error: 404 for url: https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/UNICEF,MNCH,1.0/ALB+ARM+AZE+BLR+BIH+BGR+HRV+CZE+EST+GEO+HUN+KAZ+KGZ+LVA+LTU+MNE+MKD+POL+MDA+ROU+RUS+SRB+SVK+SVN+TJK+TUR+TKM+UKR+UZB.MNCH_ABR........?startPeriod=1950&endPeriod=2050&locale=en
Dealing with indicator: NT_BW_LBW
Indicator succesfully downloaded
Dealing with indicator: CME_MRY0
Indicator succesfully downloaded
Dealing with indicator: CME_MRY0T4
Indicator succesfully downloaded
Dealing with indicator: CME_MRM0
Indicator succesfully downloaded
Dealing with indicator: MNCH_MMR
Indicator succesfully downloaded
Dealing with indicator: HVA_PMTCT_ARV_CVG
Indicator succesfully downloaded
Dealing with indicator: MNCH_PNCMOM
Indicator succesful